# Building the ML dataframe

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import xarray as xr

In [3]:
# Set max number of columns to display; default 20
pd.options.display.max_columns = 80

In [4]:
# Directory where data files will be downloaded
cwd_path = Path.cwd()
data_path = cwd_path.parent.joinpath('data')
data_push_path = cwd_path.parent.joinpath('data_to_push')

#### Load in latest wind turbine data

In [5]:
# Read in latest turbine data
# Now only 30,642 after removing 3 turbines outside bounding box
df_turbines = pd.read_pickle(data_push_path / 'df_turbines_knn_blades_haversine_elevation.pkl')
df_turbines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30642 entries, 0 to 30641
Data columns (total 30 columns):
 #   Column                                            Non-Null Count  Dtype         
---  ------                                            --------------  -----         
 0   EinheitMastrNummer                                30642 non-null  object        
 1   DatumLetzteAktualisierung                         30642 non-null  datetime64[ns]
 2   Bundesland                                        30642 non-null  object        
 3   Postleitzahl                                      30642 non-null  int64         
 4   Ort                                               30642 non-null  object        
 5   Laengengrad                                       30642 non-null  float64       
 6   Breitengrad                                       30642 non-null  float64       
 7   Registrierungsdatum                               30642 non-null  datetime64[ns]
 8   Inbetriebnahmedatum       

#### Load in SMARD data

In [159]:
# Get all SMARD CSV files, load into dataframes, and concat them

def load_SMARD_data():
    """
    Look for CSV files in the SMARD directory
    """
    list_of_dataframes = []
    for filepath in data_path.joinpath('SMARD').iterdir():
        if filepath.name.startswith('Realisierte_Erzeugung'):
            list_of_dataframes.append(pd.read_csv(filepath, sep=';'))
            
    # Check that num of rows is what I expect after concatenating dfs
    num_of_rows = 0
    for df in list_of_dataframes:
        num_of_rows += len(df)
    
    # concat dataframes
    df = pd.concat(list_of_dataframes)
    
    # Select columns to keep
    # Note: I'm using the interval start time to create the timestamps
    # ERA5 surface parameters are instantaneous so can't perfectly align anyway
    df = df[[
        'Datum',
        'Anfang',
        # 'Ende',
        'Wind Onshore [MWh] Berechnete Auflösungen'
    ]]
    
    # rename columns
    df.rename(columns={
        'Datum': 'date',
        'Anfang': 'interval_start_time', 
        # 'Ende': 'interval_end_time',
        'Wind Onshore [MWh] Berechnete Auflösungen': 'onshore_generated_mwh'
    }, inplace=True)
    
    # Handle dates and times to create unified datetime64 timestamps
    df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y')
    df['start_time_delta'] = pd.to_timedelta(df['interval_start_time'] + ':00')
    df['datetime_cet'] = df['date'] + df['start_time_delta']
    # Add CET timezone info and infer change from CET->CEST->CET, etc
    df['datetime_cet'] = df['datetime_cet'].dt.tz_localize(tz='CET', ambiguous='infer')
    
    # Drop un-needed columns
    df = df.drop(columns=['date', 'start_time_delta', 'interval_start_time'])
    # Re-arrange columns
    df = df[['datetime_cet', 'onshore_generated_mwh']]
    
    # Convert European thousands and decimal seperators in values to (US/UK) decimal full stop format
    translation_table = str.maketrans({'.': None, ',': '.'})
    df['onshore_generated_mwh'] = df['onshore_generated_mwh'].str.translate(translation_table).astype(float)
    
    print(f'Number of rows match up: {num_of_rows == len(df)}')
    
    return df.copy()

In [161]:
# Load SMARD data
df_smard = load_SMARD_data()
df_smard.info()

Number of rows match up: True
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   datetime_cet           8760 non-null   datetime64[ns, CET]
 1   onshore_generated_mwh  8760 non-null   float64            
dtypes: datetime64[ns, CET](1), float64(1)
memory usage: 137.0 KB


In [162]:
# df_smard[df_smard['datetime'] < pd.Timestamp('2022-10-30')]

In [163]:
df_smard.head(3)

,datetime_cet,onshore_generated_mwh
0,2022-01-01 00:00:00+01:00,25487.75
1,2022-01-01 01:00:00+01:00,24280.50
2,2022-01-01 02:00:00+01:00,23114.00


-----

## Begin building ML dataframe
- Start with one full year: 2022
    - Then bring in another year like 2021 and run through the same transformations and then concat along datetimeindex?
- Good resource on time-related feature engineering: https://scikit-learn.org/stable/auto_examples/applications/plot_cyclical_feature_engineering.html

In [140]:
# Year 2022
# Make time zone aware UTC?
datetime_index_utc = pd.date_range(start='2022-01-01', end='2022-12-31 23:59:59', freq='H', name='datetime_utc', tz='UTC')
datetime_index_utc

DatetimeIndex(['2022-01-01 00:00:00+00:00', '2022-01-01 01:00:00+00:00',
               '2022-01-01 02:00:00+00:00', '2022-01-01 03:00:00+00:00',
               '2022-01-01 04:00:00+00:00', '2022-01-01 05:00:00+00:00',
               '2022-01-01 06:00:00+00:00', '2022-01-01 07:00:00+00:00',
               '2022-01-01 08:00:00+00:00', '2022-01-01 09:00:00+00:00',
               ...
               '2022-12-31 14:00:00+00:00', '2022-12-31 15:00:00+00:00',
               '2022-12-31 16:00:00+00:00', '2022-12-31 17:00:00+00:00',
               '2022-12-31 18:00:00+00:00', '2022-12-31 19:00:00+00:00',
               '2022-12-31 20:00:00+00:00', '2022-12-31 21:00:00+00:00',
               '2022-12-31 22:00:00+00:00', '2022-12-31 23:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='datetime_utc', length=8760, freq='H')

In [141]:
# Create dataframe from datetimeindex, reset index and drop the duplicated datetime column
# datetime_index_utc.to_frame().reset_index(drop=True, inplace=True)

In [142]:
df = datetime_index_utc.to_frame().reset_index(drop=True)
df

,datetime_utc
0,2022-01-01 00:00:00+00:00
1,2022-01-01 01:00:00+00:00
2,2022-01-01 02:00:00+00:00
3,2022-01-01 03:00:00+00:00
4,2022-01-01 04:00:00+00:00
...,...
8755,2022-12-31 19:00:00+00:00
8756,2022-12-31 20:00:00+00:00
8757,2022-12-31 21:00:00+00:00
8758,2022-12-31 22:00:00+00:00


In [143]:
# Extract properties
df['hour'] = df['datetime_utc'].dt.hour
# 0-6 (Monday-Sunday)
df['day_of_week'] = df['datetime_utc'].dt.dayofweek
df['day_of_month'] = df['datetime_utc'].dt.day
df['month_number'] = df['datetime_utc'].dt.month

# Function to get meteorological season based on month number (1-12)
# Should I just go ahead and encode the discrete numerical value?
def get_meteorological_season(month):
    if 3 <= month <= 5:
        return 'spring'
    elif 6 <= month <= 8:
        return 'summer'
    elif 9 <= month <= 11:
        return 'autumn'
    else:
        return 'winter'
    
df['meteorological_season'] = df['month_number'].apply(get_meteorological_season)

In [144]:
df

,datetime_utc,hour,day_of_week,day_of_month,month_number,meteorological_season
0,2022-01-01 00:00:00+00:00,0,5,1,1,winter
1,2022-01-01 01:00:00+00:00,1,5,1,1,winter
2,2022-01-01 02:00:00+00:00,2,5,1,1,winter
3,2022-01-01 03:00:00+00:00,3,5,1,1,winter
4,2022-01-01 04:00:00+00:00,4,5,1,1,winter
...,...,...,...,...,...,...
8755,2022-12-31 19:00:00+00:00,19,5,31,12,winter
8756,2022-12-31 20:00:00+00:00,20,5,31,12,winter
8757,2022-12-31 21:00:00+00:00,21,5,31,12,winter
8758,2022-12-31 22:00:00+00:00,22,5,31,12,winter


-----

## Test merging SMARD data onto `df` on key datetime

In [145]:
df_merged = pd.merge(df, df_smard, left_on='datetime_utc', right_on='datetime_cet', how='left')
df_merged

,datetime_utc,hour,day_of_week,day_of_month,month_number,meteorological_season,datetime_cet,onshore_generated_mwh
0,2022-01-01 00:00:00+00:00,0,5,1,1,winter,2022-01-01 01:00:00+01:00,"24.280,5"
1,2022-01-01 01:00:00+00:00,1,5,1,1,winter,2022-01-01 02:00:00+01:00,23.114
2,2022-01-01 02:00:00+00:00,2,5,1,1,winter,2022-01-01 03:00:00+01:00,"21.998,25"
3,2022-01-01 03:00:00+00:00,3,5,1,1,winter,2022-01-01 04:00:00+01:00,20.495
4,2022-01-01 04:00:00+00:00,4,5,1,1,winter,2022-01-01 05:00:00+01:00,"19.494,5"
...,...,...,...,...,...,...,...,...
8755,2022-12-31 19:00:00+00:00,19,5,31,12,winter,2022-12-31 20:00:00+01:00,30.505
8756,2022-12-31 20:00:00+00:00,20,5,31,12,winter,2022-12-31 21:00:00+01:00,"30.768,5"
8757,2022-12-31 21:00:00+00:00,21,5,31,12,winter,2022-12-31 22:00:00+01:00,"30.490,75"
8758,2022-12-31 22:00:00+00:00,22,5,31,12,winter,2022-12-31 23:00:00+01:00,29.306


In [146]:
df_merged.head()

,datetime_utc,hour,day_of_week,day_of_month,month_number,meteorological_season,datetime_cet,onshore_generated_mwh
0,2022-01-01 00:00:00+00:00,0,5,1,1,winter,2022-01-01 01:00:00+01:00,"24.280,5"
1,2022-01-01 01:00:00+00:00,1,5,1,1,winter,2022-01-01 02:00:00+01:00,23.114
2,2022-01-01 02:00:00+00:00,2,5,1,1,winter,2022-01-01 03:00:00+01:00,"21.998,25"
3,2022-01-01 03:00:00+00:00,3,5,1,1,winter,2022-01-01 04:00:00+01:00,20.495
4,2022-01-01 04:00:00+00:00,4,5,1,1,winter,2022-01-01 05:00:00+01:00,"19.494,5"


In [147]:
df_merged.isna().sum()

datetime_utc             0
hour                     0
day_of_week              0
day_of_month             0
month_number             0
meteorological_season    0
datetime_cet             1
onshore_generated_mwh    1
dtype: int64

In [148]:
df_merged[7240:].head(15)

,datetime_utc,hour,day_of_week,day_of_month,month_number,meteorological_season,datetime_cet,onshore_generated_mwh
7240,2022-10-29 16:00:00+00:00,16,5,29,10,autumn,2022-10-29 18:00:00+02:00,"4.657,25"
7241,2022-10-29 17:00:00+00:00,17,5,29,10,autumn,2022-10-29 19:00:00+02:00,"5.988,25"
7242,2022-10-29 18:00:00+00:00,18,5,29,10,autumn,2022-10-29 20:00:00+02:00,"7.333,25"
7243,2022-10-29 19:00:00+00:00,19,5,29,10,autumn,2022-10-29 21:00:00+02:00,8.419
7244,2022-10-29 20:00:00+00:00,20,5,29,10,autumn,2022-10-29 22:00:00+02:00,8.901
7245,2022-10-29 21:00:00+00:00,21,5,29,10,autumn,2022-10-29 23:00:00+02:00,"9.505,25"
7246,2022-10-29 22:00:00+00:00,22,5,29,10,autumn,2022-10-30 00:00:00+02:00,"6.546,5"
7247,2022-10-29 23:00:00+00:00,23,5,29,10,autumn,2022-10-30 01:00:00+02:00,"9.525,5"
7248,2022-10-30 00:00:00+00:00,0,6,30,10,autumn,2022-10-30 02:00:00+02:00,"9.475,5"
7249,2022-10-30 01:00:00+00:00,1,6,30,10,autumn,2022-10-30 02:00:00+01:00,"9.311,25"


In [152]:
price = pd.read_csv(data_path.joinpath('SMARD') / 'Gro_handelspreise_202201010000_202212312359_Stunde.csv', sep=';')

In [156]:
price['Deutschland/Luxemburg [€/MWh] Originalauflösungen']

0       50,05
1       41,33
2       43,22
3       45,46
4       37,67
        ...  
8755    -1,01
8756    -1,39
8757    -1,04
8758    -1,07
8759    -0,68
Name: Deutschland/Luxemburg [€/MWh] Originalauflösungen, Length: 8760, dtype: object